Project Title: The Beverage Battle of the Neighborhoods in London, UK

Part II:

DATA AND SOLUTION DESCRIPTIONS:

First of all, I will install and import the necessary python packages, such as Beautifulsoup, wikipedia, pandas, numpy, geopy, and sklearn. Second, I will download ‘the List of areas of London’ on Wikipedia (https://en.wikipedia.org/wiki/List_of_areas_of_London) and sort the data to form the first data frame. The data frame contains data of ‘Location’, ‘Borough', 'Post town', 'PostalCode', 'Dial code', 'OS grid ref'. I will replace 'Location' with 'Neighborhod' to make it easier to understand and drop the column, 'Dial code', as it will not be used later. Next, I will convert 'OS grid ref' to latitude and longitude, create a new data frame to store the converted information, and merge this new data frame with the first one. 

After clearing the data, I will only focus on data in the post town, London, to ensure the business location derived later will be in the main area of London. I will then garner information about the most common venues in each neighborhood of the main London area using the Foursquare API. 

Subsequently, I will apply k-means clustering with k=5 to segmenting the neighborhoods based on information of the five most common venues in the respective neighborhoods. For the cluster with the highest frequency of the first three common venues being one of the following categories: coffee shop, café, pub, and bar, it will be further analyzed. Within this cluster, I will select neighborhoods showing all of the first 3 common venues being one of the aforementioned categories and present their locations using the folium package. 

Finally, I will use the ‘search for venues’ function in the Foursquare to identify coffee shops, cafés, pubs, and bars near the identified neighborhoods as the potential customers for this new beverage and ingredient supply company.  

In this session below, I will be showing data extraction and cleaning for the subsequent analysis.

In [1]:
import wikipedia
print(wikipedia.WikipediaPage(title = 'List of areas of London').summary)

This is a list of the areas of London, in alphabetical order.
London is administered by the City of London and 32 London boroughs. These boroughs are modern, having been created in 1965 and have a weaker sense of identity than their constituent "districts" (considered in speech, "parts of London" or more formally, "areas"). The boroughs were primarily formed from amalgamations of Metropolitan, County and Municipal Boroughs. These in turn were groupings of Ancient Parishes, in turn being economically large enough villages and towns which warranted a church, and before 1900 gained a Civil Parish counterpart in almost all instances. The capital had three ancient boroughs, uniting for a purpose once-subdivided parishes, London, Southwark and Kingston upon Thames. Most areas were instead towns and villages with quite steady boundaries from the Middle Ages, through the process of urbanisation and into the modern era.
Sub-districts of the districts rooted on parishes are of five types: 

form

In [11]:
import pandas as pd
import numpy as np
import geopy
import folium
from geopy.geocoders import Nominatim
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


In [12]:
from bs4 import BeautifulSoup
import requests

wikipage = 'https://en.wikipedia.org/wiki/List_of_areas_of_London'

In [13]:
page = requests.get(wikipage)
page_html = BeautifulSoup(page.text, 'lxml')
wiki_table = page_html.find('table', attrs = {'class':'wikitable sortable'})
#print(wiki_table)
row_list = wiki_table.find_all('tr')

In [14]:
header_row = row_list.pop(0)
header_th = header_row.find_all('th')
header = [el.text for el in header_th]

table_dict = {x:[] for x in header}

In [15]:
for row in row_list:
 row_td = row.find_all('td')
 for el,td in zip(header,row_td):
    table_dict[el].append(td.text)

London = pd.DataFrame(table_dict)
print(London)

                                       Location  \
0                                    Abbey Wood   
1                                         Acton   
2                                     Addington   
3                                    Addiscombe   
4                                   Albany Park   
5                              Aldborough Hatch   
6                                       Aldgate   
7                                       Aldwych   
8                                      Alperton   
9                                       Anerley   
10                                        Angel   
11                                    Aperfield   
12                                      Archway   
13                               Ardleigh Green   
14                                       Arkley   
15                                  Arnos Grove   
16                                       Balham   
17                                     Bankside   
18                             

To ensure the correct column names To remove "\n" in column 'Neighborhood'

In [16]:
London.columns=['Neighborhood', 'Borough', 'Post town', 'PostalCode', 'Dial code', 'OS grid ref']
London = London.replace('\n','', regex=True)
London['Borough'] = London['Borough'].str.replace('\d+', '')
London['Borough']=London['Borough'].str.replace("[]",'', regex=False)

London

,Neighborhood,Borough,Post town,PostalCode,Dial code,OS grid ref
0,Abbey Wood,Greenwich,LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon,CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon,CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728
5,Aldborough Hatch,Redbridge,ILFORD,IG2,020,TQ455895
6,Aldgate,City,LONDON,EC3,020,TQ334813
7,Aldwych,Westminster,LONDON,WC2,020,TQ307810
8,Alperton,Brent,WEMBLEY,HA0,020,TQ185835
9,Anerley,Bromley,LONDON,SE20,020,TQ345695


To first group based on PostCode, second to combine neighborhood names in the same PostCode, and finally to drop the duplicated rows

In [17]:
London['PostalCode'] = London[['Neighborhood','Borough','Post town','PostalCode','Dial code','OS grid ref']].groupby(['Neighborhood'])['PostalCode'].transform(lambda x: ','.join(x))
London['Borough'] = London[['Neighborhood','Borough','Post town','PostalCode','Dial code','OS grid ref']].groupby(['Neighborhood'])['Borough'].transform(lambda x: ','.join(x))
London=London[['Neighborhood','Borough','Post town','PostalCode','Dial code','OS grid ref']].drop_duplicates(subset='Neighborhood')
London=London.drop(['Dial code'], axis =1)

In [18]:
London.to_csv('London.csv', index=False)
London.shape

(527, 5)

Using the folllowing link to convert 'OS grid ref' in London data frame to 'latitude' and 'longitude' and save the file as 'London_Geospatial_Coordinates.csv'
https://gridreferencefinder.com/batchConvert/batchConvert.php

In [20]:
coord=pd.read_csv('London_Geospatial_Coordinates.csv')
coord.shape

(527, 5)

In [22]:
combin = pd.merge(London, coord)
combin

,Neighborhood,Borough,Post town,PostalCode,OS grid ref,X,Y,Lat,Lng
0,Abbey Wood,Greenwich,LONDON,SE2,TQ465785,546500.0,178500.0,51.486481,0.108592
1,Acton,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4",TQ205805,520500.0,180500.0,51.510588,-0.264989
2,Addington,Croydon,CROYDON,CR0,TQ375645,537500.0,164500.0,51.362931,-0.026374
3,Addiscombe,Croydon,CROYDON,CR0,TQ345665,534500.0,166500.0,51.381622,-0.068682
4,Addiscombe,Croydon,CROYDON,CR0,TQ345665,534500.0,166500.0,51.381622,-0.068682
5,Angel,Islington,LONDON,"EC1, N1",TQ345665,534500.0,166500.0,51.381622,-0.068682
6,Angel,Islington,LONDON,"EC1, N1",TQ345665,534500.0,166500.0,51.381622,-0.068682
7,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",TQ478728,547800.0,172800.0,51.434926,0.124921
8,Aldborough Hatch,Redbridge,ILFORD,IG2,TQ455895,545500.0,189500.0,51.585578,0.098742
9,Aldgate,City,LONDON,EC3,TQ334813,533400.0,181300.0,51.514882,-0.078905


In [23]:
combin.to_csv('combin.csv', index=False)

In [24]:
Lon_data = combin[combin['Post town'] == 'LONDON'].reset_index(drop=True)
Lon_data['PostalCode'] = Lon_data[['Neighborhood','Borough','Post town','PostalCode','OS grid ref','X','Y','Lat','Lng']].groupby(['Neighborhood'])['PostalCode'].transform(lambda x: ','.join(x))
Lon_data['Borough'] = Lon_data[['Neighborhood','Borough','Post town','PostalCode','OS grid ref','X','Y','Lat','Lng']].groupby(['Neighborhood'])['Borough'].transform(lambda x: ','.join(x))
Lon_data=Lon_data[['Neighborhood','Borough','Post town','PostalCode', 'OS grid ref', 'X','Y','Lat','Lng']].drop_duplicates(subset='Neighborhood')
Lon_data.head()
Lon_data.shape
Lon_data.to_csv('Lon_data.csv', index=False)